In [1]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer 
from sklearn.feature_extraction.text import TfidfVectorizer
import re
from scipy.sparse import hstack

data_train = pd.read_csv('salary-train.csv')
data_test = pd.read_csv('salary-test-mini.csv')

In [2]:
# Приведение текстов к нижнему регистру
data_train['FullDescription'] = [x.lower() for x in data_train['FullDescription']]
data_train['LocationNormalized'] = [x.lower() for x in data_train['LocationNormalized']]

data_test['FullDescription'] = [x.lower() for x in data_test['FullDescription']]
data_test['LocationNormalized'] = [x.lower() for x in data_test['LocationNormalized']]


# Замена символов не являющихся цифрами и буквами на пробелы
data_train['FullDescription'] = data_train['FullDescription'].replace('[^a-zA-Z0-9]', ' ', regex = True)
data_test['FullDescription'] = data_test['FullDescription'].replace('[^a-zA-Z0-9]', ' ', regex = True)

X_train_text = data_train['FullDescription']
X_test_text = data_test['FullDescription']

# Перевод текстов в признаки через матрицу TF-IDF
tf_idf_vectorizer = TfidfVectorizer(min_df=5)
X_train_text = tf_idf_vectorizer.fit_transform(X_train_text)
X_test_text = tf_idf_vectorizer.transform(X_test_text)

# Замена NaN
data_train['LocationNormalized'].fillna('nan', inplace=True)
data_train['ContractTime'].fillna('nan', inplace=True)

data_test['LocationNormalized'].fillna('nan', inplace=True)
data_test['ContractTime'].fillna('nan', inplace=True)

# one-hot кодирование категориальных признаков
enc = DictVectorizer()
X_train_categ = enc.fit_transform(data_train[['LocationNormalized', 'ContractTime']].to_dict('records'))
X_test_categ = enc.transform(data_test[['LocationNormalized', 'ContractTime']].to_dict('records'))

X_train = hstack([X_train_text, X_train_categ])
Y_train = data_train['SalaryNormalized']

X_test = hstack([X_test_text, X_test_categ])
Y_test = data_test['SalaryNormalized']

In [3]:
from sklearn.linear_model import Ridge
clf = Ridge(alpha=1, random_state=241)
clf.fit(X_train, Y_train)

Ridge(alpha=1, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=241, solver='auto', tol=0.001)

In [5]:
prediction = clf.predict(X_test)

In [8]:
for pred in prediction:
    print('%.2f', % pred)

SyntaxError: invalid syntax (<ipython-input-8-5468e6363a98>, line 2)